In [1]:
# install python-dotenv, langchain, langchain-openai, google-cloud-bigquery, gradio

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from google.cloud import bigquery
from google.oauth2 import service_account
import gradio as gr

load_dotenv()

True

In [2]:
# Set up BigQuery Account Credentials

service_account_path = '.\gbqkey.json'
dataset_id = 'maxs-bi-project.maxbiproject'
table_id = "events_20210131"
credentials = service_account.Credentials.from_service_account_file(service_account_path)
gbq_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [3]:
# Define a function to build a schema decription in natural language to feed to the model, inclduing nested fields

def build_schema_desc(fields, prefix=""):
    desc = []
    for f in fields:
        d = f"{prefix}- Name: {f.name}, Type: {f.field_type}, Mode: {f.mode}"
        desc.append(d)
        if f.field_type == 'RECORD':
            sub_desc = build_schema_desc(f.fields, prefix + "    ")
            desc.extend(sub_desc)
    return desc

In [4]:
# Define function a to fetch a schema from a BigQuery Table based on dataset_id, my BigQuery client with my service account credentials, and the table_id
# If your want to query a different table simply modify the dataset_id or table_id accordingly

def fetch_schemas(dataset_id, client, table_id):
    schemas = []
    simple_table_list = []

    tables = client.list_tables(dataset_id)
    for table in tables:
        if table_id in table.table_id:
            ref = client.get_table(table)

            simple_table_list.append(f"- {ref.project}.{ref.dataset_id}.{ref.table_id}")

            schema_desc = [f"Schema for {table.table_id}:"]
            schema_desc += build_schema_desc(ref.schema)
            schema_desc.append("")  # For newline

            schemas += schema_desc

    return "\n".join(simple_table_list) + "\n\n" + "\n".join(schemas)

In [5]:
fetch_schemas(dataset_id,gbq_client,table_id)

'- maxs-bi-project.maxbiproject.events_20210131\n\nSchema for events_20210131:\n- Name: event_date, Type: STRING, Mode: NULLABLE\n- Name: event_timestamp, Type: INTEGER, Mode: NULLABLE\n- Name: event_name, Type: STRING, Mode: NULLABLE\n- Name: event_params, Type: RECORD, Mode: REPEATED\n    - Name: key, Type: STRING, Mode: NULLABLE\n    - Name: value, Type: RECORD, Mode: NULLABLE\n        - Name: string_value, Type: STRING, Mode: NULLABLE\n        - Name: int_value, Type: INTEGER, Mode: NULLABLE\n        - Name: float_value, Type: FLOAT, Mode: NULLABLE\n        - Name: double_value, Type: FLOAT, Mode: NULLABLE\n- Name: event_previous_timestamp, Type: INTEGER, Mode: NULLABLE\n- Name: event_value_in_usd, Type: FLOAT, Mode: NULLABLE\n- Name: event_bundle_sequence_id, Type: INTEGER, Mode: NULLABLE\n- Name: event_server_timestamp_offset, Type: INTEGER, Mode: NULLABLE\n- Name: user_id, Type: STRING, Mode: NULLABLE\n- Name: user_pseudo_id, Type: STRING, Mode: NULLABLE\n- Name: privacy_info, T

## Setup SQL Chain

In [6]:
# Write a prompt that uses the database schema to build a SQL Query based on the user's question
template = """
Based on the BigQuery schema below, write a SQL query (only return clean SQL no quotes or text)  that answers the question/request. Write main query ALIAS in short simple capital letters. Remember to UNNEST repeated records and CAST timestamps/dates in the right format when comparing or subtracting:
schema: {schema}
Question: {question}
SQL Query:
"""
prompt = ChatPromptTemplate.from_template(template)

In [7]:
# Use Chat GPT 4 as the model
llm = ChatOpenAI(model_name="gpt-4")

In [8]:
# Define a function that uses the fetch schemas function with a single input
def get_schema(_):
    return fetch_schemas(dataset_id,gbq_client,table_id)

In [9]:
# Create a chain that 
sql_chain = (RunnablePassthrough.assign(schema=get_schema) | prompt | llm.bind(stop=["\nSQLResult:"])  | StrOutputParser())

In [10]:
# Ask the database a question in natural language!
response=sql_chain.invoke({"question": 'What are the five most purchased items?'})
gbq_client.query(response).to_dataframe() # Running the query throught the BigQuery table

,item_name,total_purchased
0,Google Twill Cap Charcoal,55
1,Google Keychain,40
2,Google Laptop and Cell Phone Stickers,26
3,Google Bot (Multicolored),25
4,YouTube Twill Sandwich Cap Black,22


## Create Gradio App

In [11]:
# Initialize an empty string to hold the dataframes
dataframes = ''

# Function to handle Gradio input and interact with the SQL chain
def ask_question(question):
    global dataframes
    response = sql_chain.invoke({"question": question})
    df = gbq_client.query(response).to_dataframe()
    dataframes += f"Question: {question}\nDataFrame:\n{df.to_string()}\n\n" # Append the question and its DataFrame (converted to string) to 'dataframes'
    
    return df

In [12]:
# Create a second prompt to help analyze the data. Tell it to take in the string of dataframes, chat message history, and a user input.
prompt2 = ChatPromptTemplate.from_messages(
    [        
        ("system", '''
You are Kregg, a friendly and engaging business analytics assistant. Your goal is to make data querying and business insights fun and conversational for the users. Present information in an entertaining and approachable manner using humor, casual language, and relatable analogies. Be concise and clear, providing insightful answers and guidance to help users make informed decisions. Here are some examples to guide your responses:

Examples:
User Query: "What were our total sales last quarter?"

Kregg's Response: "Ah, the age-old question of how much moolah we made! Hold tight while I crunch the numbers... Voilà! Our total sales last quarter were $X. Ka-ching!"
User Query: "How many new customers did we acquire last month?"

Kregg's Response: "Let's see how many new friends joined our party last month! Drum roll, please... We welcomed Y new customers. Woohoo!"
User Query: "Which products are low in stock?"

Kregg's Response: "Time to check our shelves! It looks like these products are running low: [Product A], [Product B], and [Product C]. Better restock before they vanish like hotcakes!"
User Query: "What was the ROI of our last marketing campaign?"

Kregg's Response: "Let's decode the mystery of our marketing magic. The ROI of our last campaign was Z%. Not too shabby, right?"
User Query: "Generate a report on our monthly revenue."

Kregg's Response: "Sure thing! I'll whip up a fancy report on our monthly revenue. Here it is, all nice and neat. Enjoy the read!"
Guidelines:
Accuracy: Ensure that the information provided is accurate.
Clarity: Present information in a clear and understandable manner.
Engagement: Use humor, casual language, and analogies to make the conversation lively and enjoyable.
Conciseness: Keep responses concise and to the point, providing essential information without overwhelming the user.
        '''),
        ("system", "{dataframes}"),  # Injecting dataframe information as a template variable
        ("human", "{input}")
    ]
)

In [13]:
convo_chain = prompt2 | llm

In [14]:
def convo(question):
    response = convo_chain.invoke({"input": question, "dataframes": dataframes})
    return response.content

In [16]:
# Build a Gradio app with the two ChatBots
with gr.Blocks() as blocks:
    gr.Markdown("# Meet Kregg!")
    gr.Markdown("### The AI BI analyst who can facilitate a chat between you andd your data! Get insights from your data by asking questions in natural language. Start by asking a querying your database and then ask Kregg to analyze your data for you.")
    gr.Markdown("#### Suggested Query Questions:")
    gr.Markdown(
        """
        - "What are the five most purchased products?"
        - "What are the most profitable traffic sources?"
        - "Which users have the best lifetime value?"
        """
    )
    gr.Markdown("#### Suggested Analysis Questions:")
    gr.Markdown(
        """
        - "What insights can you tell me about my data?"
        - "Generate a report based on this data."
        - "Come up with a slide deck based on this data."
        """
    )

    gr.Interface(fn=ask_question, inputs="text", outputs="dataframe", title="Ask Your Question and Let Kregg Run the Query")
    gr.Interface(fn=convo, inputs="text", outputs="text", title="Analyze Your Data in Natural Language")

blocks.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
